In [1]:
from tensorflow import keras
import pandas as pd
from sklearn.preprocessing import StandardScaler

import plotly.graph_objects as go
import numpy as np
import tensorflow as tf

np.random.seed(1)
tf.random.set_seed(1)

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout, RepeatVector, TimeDistributed

print('Tensorflow version: ', tf.__version__)

Tensorflow version:  2.8.0


In [2]:
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA

In [3]:
pip install plotly==5.5.0

Note: you may need to restart the kernel to use updated packages.


In [4]:
df = pd.read_csv('weather.csv')
df['Timestamp'] = pd.to_datetime(df['Timestamp'])
df = df.sort_values(by='Timestamp',ascending=True)

In [5]:
df.head()

,Timestamp,Min_Windspeed1,Max_Windspeed1,Avg_Windspeed1,Var_Windspeed1,Min_Windspeed2,Max_Windspeed2,Avg_Windspeed2,Var_Windspeed2,Min_Winddirection2,...,Anemometer2_CorrOffset,DistanceAirPress,AirRessureSensorZeroOffset,Anemometer1_Avg_Freq,Anemometer2_Avg_Freq,Pressure_Avg_Freq,Year,Month,Day,Hour
0,2016-01-01 00:00:00+00:00,3.7,6.0,5.1,0.21,3.8,6.0,5.1,0.22,236,...,0,0,600,98,99,418,2016,1,1,0
1,2016-01-01 00:10:00+00:00,4.1,6.0,5.1,0.09,4.1,6.0,5.2,0.10,236,...,0,0,600,99,101,418,2016,1,1,0
2,2016-01-01 00:20:00+00:00,4.5,6.7,5.7,0.26,4.4,6.8,5.8,0.30,236,...,0,0,600,111,113,418,2016,1,1,0
3,2016-01-01 00:30:00+00:00,5.1,7.0,6.3,0.11,5.1,7.1,6.4,0.12,236,...,0,0,600,122,125,418,2016,1,1,0
4,2016-01-01 00:40:00+00:00,4.7,7.3,6.2,0.27,4.9,7.4,6.3,0.27,236,...,0,0,600,121,123,417,2016,1,1,0


In [6]:
df['Timestamp'] = pd.to_datetime(df['Timestamp'])

In [7]:
df = df[['Timestamp','Min_Windspeed1']]

In [8]:
df['Timestamp'].min(), df['Timestamp'].max()

(Timestamp('2016-01-01 00:00:00+0000', tz='UTC'),
 Timestamp('2016-12-31 23:50:00+0000', tz='UTC'))

In [10]:
scaled = df.drop(columns = ['Timestamp'])

In [11]:
from sklearn.preprocessing import StandardScaler
scalar = StandardScaler()
df_scaled = scalar.fit_transform(scaled)
df_scaled

array([[-0.23771023],
       [-0.09139006],
       [ 0.05493011],
       ...,
       [-0.23771023],
       [ 0.05493011],
       [ 0.09151015]])

In [12]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=df_scaled['Timestamp'], y=df_scaled['Min_Windspeed1'], name='Min_Windspeed1'))
fig.update_layout(showlegend=True, title='Max_Humidity 2016')
fig.show()

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [ ]:
df_scaled.plot()

In [ ]:
df_scaled.set_index('Timestamp',inplace=True)
df_scaled.index=pd.to_datetime(df_scaled.index)
#drop null values
df_scaled.dropna(inplace=True)
df_scaled.plot()

In [ ]:
df_scaled

In [ ]:
train, test = df_scaled.loc[df_scaled['Timestamp'] <= '2016-09-01 00:00:00+0000'], df_scaled.loc[df_scaled['Timestamp'] > '2016-09-01 00:00:00+0000']

In [ ]:
train.tail()

In [ ]:
test.head()

In [ ]:
train.shape,test.shape

In [ ]:
scaler = StandardScaler()
scaler = scaler.fit(train[['Max_Humidity']])

train['Max_Humidity'] = scaler.transform(train[['Max_Humidity']])
test['Max_Humidity'] = scaler.transform(test[['Max_Humidity']])



In [ ]:
train

In [ ]:
TIME_STEPS=30

def create_sequences(X, y, time_steps=TIME_STEPS):
    Xs, ys = [], []
    for i in range(len(X)-time_steps):
        Xs.append(X.iloc[i:(i+time_steps)].values)
        ys.append(y.iloc[i+time_steps])
    
    return np.array(Xs), np.array(ys)

X_train, y_train = create_sequences(train[['Max_Humidity']], train['Max_Humidity'])
X_test, y_test = create_sequences(test[['Max_Humidity']], test['Max_Humidity'])

In [ ]:
print(f'Training shape: {X_train.shape}')

print(f'Testing shape: {X_test.shape}')

In [ ]:

model = Sequential()

# model.add(keras.layers.LSTM(64,input_shape=inputshape))
model.add(LSTM(128, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dropout(rate=0.2))
model.add(RepeatVector(X_train.shape[1]))
model.add(LSTM(128, return_sequences=True))
model.add(Dropout(rate=0.2))
model.add(TimeDistributed(Dense(X_train.shape[2])))
model.compile(optimizer='adam', loss='mae')
model.summary()

In [ ]:
history = model.fit(X_train, y_train, epochs=100, batch_size=32, validation_split=0.1,
                    callbacks=[keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, mode='min')], shuffle=False)

In [ ]:
import matplotlib.pyplot as plt

plt.plot(history.history['loss'], label='Training loss')
plt.plot(history.history['val_loss'], label='Validation loss')
plt.legend();

In [ ]:
model.evaluate(X_test, y_test)

In [ ]:
X_train_pred = model.predict(X_train, verbose=0)
train_mae_loss = np.mean(np.abs(X_train_pred - X_train), axis=1)

plt.hist(train_mae_loss, bins=50)
plt.xlabel('Train MAE loss')
plt.ylabel('Number of Samples');
threshold = np.max(train_mae_loss)-0.8
# threshold = 1.75
print(f'Reconstruction error threshold: {threshold}')

In [ ]:
X_test_pred = model.predict(X_test, verbose=0)
test_mae_loss = np.mean(np.abs(X_test_pred-X_test), axis=1)

plt.hist(test_mae_loss, bins=50)
plt.xlabel('Test MAE loss')
plt.ylabel('Number of samples');

In [ ]:
test_score_df = pd.DataFrame(test[TIME_STEPS:])
test_score_df['loss'] = test_mae_loss
test_score_df['threshold'] = threshold
test_score_df['anomaly'] = test_score_df['loss'] > test_score_df['threshold']
test_score_df['Max_Humidity'] = test[TIME_STEPS:]['Max_Humidity']

In [ ]:
test_score_df.tail()

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=test_score_df['Timestamp'], y=test_score_df['loss'], name='Test loss'))
fig.add_trace(go.Scatter(x=test_score_df['Timestamp'], y=test_score_df['threshold'], name='Threshold'))
fig.update_layout(showlegend=True, title='Test loss vs. Threshold')
fig.show()

In [ ]:
anomalies = test_score_df.loc[test_score_df['anomaly'] == True]
anomalies.head()

In [ ]:
test_score_df

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=test_score_df['Timestamp'], y=scaler.inverse_transform(test_score_df['Max_Humidity']), name='Max_Humidity'))
fig.add_trace(go.Scatter(x=anomalies['Timestamp'], y=scaler.inverse_transform(anomalies['Max_Humidity']), mode='markers', name='Anomaly'))
fig.update_layout(showlegend=True, title='Detected anomalies')
fig.show()